In [1]:
import datetime
import pandas as pd
import matplotlib as mpl

In [2]:
%matplotlib inline
mpl.style.use('classic')

# Set input and output filenames

In [3]:
cleaned_ss = './raw_data/2023-01-22-ss.cleaned.csv.gz'

cullpdb_input = './raw_data/cullpdb_pc30.0_res0.0-2.5_len40-10000_R0.3_Xray_d2022_12_17_chains15208.gz'
intersect_output = './raw_data/2023-01-22-pdb-intersect-pisces_pc30_r2.5.csv'

# Clean PISCES data

PISCES: https://academic.oup.com/bioinformatics/article/19/12/1589/258419

PISCES datasets are composed of lists of PDB accession numbers and chain identifiers which have been curated to to contain chains that are relatively non-redundant (percent identity), of particular resolution cutoffs, and composed of X-ray, cryo-electron microscopy, and/or NMR structures.  These are used here to limit the content of the downloaded, .csv transformed, and cleaned secondary structure information from RCSB-PDB (ss.txt and ss.cleaned.csv).

In [4]:
# Change the filename to refer to the specific cullpdb* file of interest
# with appropriate cutoff criteria

pisces_df = pd.read_csv(cullpdb_input,  sep=r'[\t ]+', engine='python')

In [5]:
pisces_df.head(2)

,PDBchain,len,method,resol,rfac,freerfac
0,5D8VA,83,XRAY,0.48,0.072,0.078
1,3NIRA,46,XRAY,0.48,0.127,NaN


In [6]:
pisces_df.shape

(15208, 6)

**Note that in the original dataset from 2018, the column names were:***

```IDs length Exptl. resolution R-factor FreeRvalue```

**The following code has been changed to refer to the new column names shown above.***

In [7]:
# make sure IDs values are all unique
assert pisces_df.PDBchain.unique().shape[0] == pisces_df.shape[0]

In [8]:
%%time
# For explanantion of chain code: http://dunbrack.fccc.edu/Guoli/pisces_download.php
pdb_id_chain_ids = pisces_df.PDBchain.apply(
    lambda s: pd.Series([s, s[:4], s[4]], index=['PDBchain', 'pdb_id', 'chain_code']))

CPU times: user 2.78 s, sys: 35.2 ms, total: 2.82 s
Wall time: 2.83 s


In [9]:
pdb_id_chain_ids.head()

# The column name PDBchain was previously IDs

,PDBchain,pdb_id,chain_code
0,5D8VA,5D8V,A
1,3NIRA,3NIR,A
2,5NW3A,5NW3,A
3,1UCSA,1UCS,A
4,3X2MA,3X2M,A


In [10]:
# replaced PDBchain column with (pdb_id, chain_code) columns
out_df = pisces_df.merge(pdb_id_chain_ids, on='PDBchain').drop('PDBchain', axis=1)

In [11]:
out_df.head()

,len,method,resol,rfac,freerfac,pdb_id,chain_code
0,83,XRAY,0.48,0.072,0.078,5D8V,A
1,46,XRAY,0.48,0.127,NaN,3NIR,A
2,54,XRAY,0.59,0.135,0.146,5NW3,A
3,64,XRAY,0.62,0.139,0.155,1UCS,A
4,180,XRAY,0.64,0.122,0.129,3X2M,A


# Intersect with pdb

The ss.cleaned.csv.gz file is generated using Clean-pdb-ss-csv.ipynb

In [ ]:
%%time
adf = pd.read_csv(cleaned_ss, dtype=str)

In [ ]:
adf['len'] = adf['len'].astype(int)

In [ ]:
adf.shape

In [ ]:
adf.head()

In [ ]:
%time bdf = adf.merge(out_df, on=['pdb_id', 'chain_code'])

In [ ]:
bdf.shape

In [ ]:
bdf.head(2)

In [ ]:
bdf.describe()

### Check that lengths from PDB ss.txt and cullpdb match

In [ ]:
bdf.query('len_x != len_y')

#### They don't match, so check which (if either) is correct

In [ ]:
q = bdf.query('len_x != len_y')

print("seq = ", [len(s) for s in q.seq])
print("sst8 = ", [len(s) for s in q.sst8])
print('sst3 = ', [len(s) for s in q.sst3])

PDB ss.txt is accurate, which is consistent with the original dataset findings from 2018.

Original note:

Verified that the `ss.txt.gz` from pdb is more update-to-date (see [fasta](https://www.rcsb.org/pdb/download/viewFastaFiles.do?structureIdList=5LTR&compressionType=uncompressed)), so drop `length` column

In [ ]:
bdf.len_x.plot.hist(bins=20)

In [ ]:
bdf.head(2)

# Get a sense of sst distribution

In [ ]:
sst8_sr = pd.Series(list(''.join(bdf.sst8.values.tolist())))

In [ ]:
sst8_sr.value_counts().plot.bar()

In [ ]:
sst3_sr = pd.Series(list(''.join(bdf.sst3.values.tolist())))

In [ ]:
sst3_sr.value_counts().plot.bar()

# Get a sense of aa distribution

In [ ]:
aa_sr = sst_sr = pd.Series(list(''.join(bdf.seq.values.tolist())))

In [ ]:
aa_sr.value_counts().plot.bar()

In [ ]:
bdf.to_csv(intersect_output, index=False)